In [1]:
import torch
from torch import nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open('../Data/shakespeare.txt', 'r', encoding='utf8') as f:
    text = f.read()

In [3]:
type(text)

str

In [4]:
print(text[:100])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose mi


In [5]:
len(text)

5445609

In [6]:
all_characters = set(text)

In [7]:
all_characters

{'\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}'}

In [8]:
len(all_characters)

84

In [9]:
# num --> letter
decoder = dict(enumerate(all_characters))
decoder

{0: 'd',
 1: '8',
 2: 'f',
 3: '6',
 4: 'S',
 5: 'b',
 6: ')',
 7: ',',
 8: 'c',
 9: 'q',
 10: '"',
 11: 'B',
 12: ':',
 13: 'E',
 14: 'i',
 15: 'O',
 16: 'u',
 17: 'H',
 18: '7',
 19: 'W',
 20: '|',
 21: 'F',
 22: 'p',
 23: '_',
 24: 'o',
 25: ']',
 26: 'z',
 27: 'Z',
 28: 'k',
 29: '1',
 30: 'A',
 31: '3',
 32: 'L',
 33: '5',
 34: 'Q',
 35: 'C',
 36: '!',
 37: 'j',
 38: 'M',
 39: 'l',
 40: 'I',
 41: '&',
 42: 's',
 43: ';',
 44: 'v',
 45: 'K',
 46: 'e',
 47: '2',
 48: ' ',
 49: 'T',
 50: '9',
 51: '<',
 52: '-',
 53: 'V',
 54: '`',
 55: 'g',
 56: 'w',
 57: 'a',
 58: 'G',
 59: '?',
 60: 'J',
 61: 'D',
 62: 'h',
 63: '4',
 64: 'X',
 65: 't',
 66: '\n',
 67: "'",
 68: '0',
 69: 'r',
 70: 'Y',
 71: 'n',
 72: 'm',
 73: '>',
 74: 'N',
 75: 'R',
 76: '.',
 77: 'x',
 78: '(',
 79: 'U',
 80: '[',
 81: '}',
 82: 'P',
 83: 'y'}

In [10]:
# letter --> num
encoder = {char: ind for ind, char in decoder.items()}

In [11]:
encoder

{'d': 0,
 '8': 1,
 'f': 2,
 '6': 3,
 'S': 4,
 'b': 5,
 ')': 6,
 ',': 7,
 'c': 8,
 'q': 9,
 '"': 10,
 'B': 11,
 ':': 12,
 'E': 13,
 'i': 14,
 'O': 15,
 'u': 16,
 'H': 17,
 '7': 18,
 'W': 19,
 '|': 20,
 'F': 21,
 'p': 22,
 '_': 23,
 'o': 24,
 ']': 25,
 'z': 26,
 'Z': 27,
 'k': 28,
 '1': 29,
 'A': 30,
 '3': 31,
 'L': 32,
 '5': 33,
 'Q': 34,
 'C': 35,
 '!': 36,
 'j': 37,
 'M': 38,
 'l': 39,
 'I': 40,
 '&': 41,
 's': 42,
 ';': 43,
 'v': 44,
 'K': 45,
 'e': 46,
 '2': 47,
 ' ': 48,
 'T': 49,
 '9': 50,
 '<': 51,
 '-': 52,
 'V': 53,
 '`': 54,
 'g': 55,
 'w': 56,
 'a': 57,
 'G': 58,
 '?': 59,
 'J': 60,
 'D': 61,
 'h': 62,
 '4': 63,
 'X': 64,
 't': 65,
 '\n': 66,
 "'": 67,
 '0': 68,
 'r': 69,
 'Y': 70,
 'n': 71,
 'm': 72,
 '>': 73,
 'N': 74,
 'R': 75,
 '.': 76,
 'x': 77,
 '(': 78,
 'U': 79,
 '[': 80,
 '}': 81,
 'P': 82,
 'y': 83}

In [12]:
encoded_text = np.array([encoder[char] for char in text])

In [13]:
encoded_text[:500]

array([66, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48,
       48, 48, 48, 48, 48, 29, 66, 48, 48, 21, 69, 24, 72, 48,  2, 57, 14,
       69, 46, 42, 65, 48,  8, 69, 46, 57, 65, 16, 69, 46, 42, 48, 56, 46,
       48,  0, 46, 42, 14, 69, 46, 48, 14, 71,  8, 69, 46, 57, 42, 46,  7,
       66, 48, 48, 49, 62, 57, 65, 48, 65, 62, 46, 69, 46,  5, 83, 48,  5,
       46, 57, 16, 65, 83, 67, 42, 48, 69, 24, 42, 46, 48, 72, 14, 55, 62,
       65, 48, 71, 46, 44, 46, 69, 48,  0, 14, 46,  7, 66, 48, 48, 11, 16,
       65, 48, 57, 42, 48, 65, 62, 46, 48, 69, 14, 22, 46, 69, 48, 42, 62,
       24, 16, 39,  0, 48,  5, 83, 48, 65, 14, 72, 46, 48,  0, 46,  8, 46,
       57, 42, 46,  7, 66, 48, 48, 17, 14, 42, 48, 65, 46, 71,  0, 46, 69,
       48, 62, 46, 14, 69, 48, 72, 14, 55, 62, 65, 48,  5, 46, 57, 69, 48,
       62, 14, 42, 48, 72, 46, 72, 24, 69, 83, 12, 66, 48, 48, 11, 16, 65,
       48, 65, 62, 24, 16, 48,  8, 24, 71, 65, 69, 57,  8, 65, 46,  0, 48,
       65, 24, 48, 65, 62

In [14]:
decoder[27]

'Z'

In [15]:
def one_hot_encoder(encoded_text, num_uni_chars):
    
    # encoded_text --> batch of encoded text
    # num_uni_chars --> len(set(text))
    
    one_hot = np.zeros((encoded_text.size, num_uni_chars))
    
    one_hot = one_hot.astype(np.float32)
    
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()]  = 1.0
    
    one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))
    
    return one_hot

In [16]:
arr = np.array([1, 2, 0])
arr

array([1, 2, 0])

In [17]:
one_hot_encoder(arr, 3)

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

In [18]:
example_text = np.arange(10)

In [19]:
example_text

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [20]:
example_text.reshape((5,-1))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [21]:
def generate_batches(encoded_text, samp_per_batch=10, seq_len=50):
    
    # X : encoded text of legth seq_len
    # Y : encoded text shifted by one
    
    # how many chars per batch?
    char_per_batch = samp_per_batch * seq_len
    
    # how many batches can we make, given the len of the encoded text?
    num_batches_avail = int(len(encoded_text)/char_per_batch)
    
    # Cut off the end of the encoded text, that won't fit evenly into a batch
    encoded_text = encoded_text[:num_batches_avail * char_per_batch]

    encoded_text = encoded_text.reshape((samp_per_batch, -1))
    
    for n in range(0, encoded_text.shape[1], seq_len):
        
        x = encoded_text[:, n:n+seq_len]
        
        # zeros array to the same shape as x
        y = np.zeros_like(x)
        
        try:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, n+seq_len]
        except:
            y[:, :-1] = x[:, 1:]
            y[:,-1] = encoded_text[:, 0]
        
        yield x,y

In [22]:
sample_text = encoded_text[:20]
sample_text

array([66, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48,
       48, 48, 48])

In [23]:
batch_generator = generate_batches(sample_text, samp_per_batch=2, seq_len=5)

In [24]:
x,y = next(batch_generator)

In [25]:
x

array([[66, 48, 48, 48, 48],
       [48, 48, 48, 48, 48]])

In [26]:
y

array([[48, 48, 48, 48, 48],
       [48, 48, 48, 48, 48]])

In [27]:
class CharModel(nn.Module):
    
    def __init__(self, all_chars, num_hidden=256, num_layers=4, drop_prob=0.5, use_gpu=False):
        
        super().__init__()
        
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu
        
        self.all_chars = all_chars
        self.decoder = dict(enumerate(all_chars))
        self.encoder = {char: ind for ind, char in decoder.items()}
        
        self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))
    
    def forward(self, x, hidden):
        
        lstm_output, hidden = self.lstm(x, hidden)
        
        drop_output = self.dropout(lstm_output)
        
        drop_output = drop_output.contiguous().view(-1, self.num_hidden)
        
        final_out = self.fc_linear(drop_output)
        
        return final_out, hidden
    
    def hidden_state(self, batch_size):
        
        if self.use_gpu:
            
            hidden = (torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda(), torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda())
        else:
            hidden = (torch.zeros(self.num_layers, batch_size, self.num_hidden), torch.zeros(self.num_layers, batch_size, self.num_hidden))
        
        return hidden
    

In [28]:
model = CharModel(all_chars=all_characters,
                  num_hidden=512,
                  num_layers=3,
                  drop_prob=0.5,
                  use_gpu=False)

In [29]:
total_param = []

for p in model.parameters():
    total_param.append(int(p.numel()))

sum(total_param)

5470292

In [30]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [31]:
train_percent = 0.9
train_ind = int(len(encoded_text)*train_percent)

train_data = encoded_text[:train_ind]
val_data = encoded_text[train_ind:]

len(train_data), len(val_data)

(4901048, 544561)

In [32]:
# VARIABLES

epochs = 2
batch_size = 100

seq_len = 100

tracker = 0
num_char = max(encoded_text) + 1


In [33]:
model.train()

if model.use_gpu:
    model.cuda()
    
for i in range(epochs):
    
    hidden = model.hidden_state(batch_size)
    
    for x,y in generate_batches(train_data, batch_size, seq_len):
    
        tracker += 1
        
        x = one_hot_encoder(x, num_char)
        
        inputs = torch.from_numpy(x)
        targets = torch.from_numpy(y)
        
        if model.use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()
        
        hidden = tuple([state.data for state in hidden])
        
        model.zero_grad()
            
        lstm_output, hidden = model.forward(inputs, hidden)
        loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())
        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        
        optimizer.step()
        
        if tracker % 25 == 0:
            
            val_hidden = model.hidden_state(batch_size)
            val_losses = []
            model.eval()
            
            for x,y in generate_batches(val_data, batch_size, seq_len):
                
                x = one_hot_encoder(x, num_char)
        
                inputs = torch.from_numpy(x)
                targets = torch.from_numpy(y)

                if model.use_gpu:
                    inputs = inputs.cuda()
                    targets = targets.cuda()
                
                val_hidden = tuple([state.data for state in val_hidden])
                
                lstm_output, val_hidden = model.forward(inputs, val_hidden)
                val_loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())
                val_losses.append(val_loss.item())
            
            model.train()
            
            print(f"EPOCH: {i} step: {tracker} VAL LOSS: {val_loss.item()}")
    

KeyboardInterrupt: 

In [37]:
model_name = "hidden512_layers3_shakes_saul.net"

In [39]:
torch.save(model.state_dict(), model_name)